In [ ]:
import numpy as np
import cv2
import shap
import torch
import torchvision
import matplotlib.pyplot as plt

from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image, preprocess_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from PIL import Image


print(f'Torch-Version {torch.__version__}')
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'DEVICE: {DEVICE}')

#### **Grad-CAM visualisation**

Use the Grad-CAM XAI tool to analyse the model attention and generate visualisation heatmaps.

In [ ]:
# Load trained base models
new_effici_model = torch.load('saved_model/pre-trained_model.pth') # Refer to the reload pre-trained base model in ensemble learning file
new_resnet_model = torch.load('saved_model/pre-trained_model.pth')
new_mobile_model = torch.load('saved_model/pre-trained_model.pth')

In [ ]:
# Select the target sample 
image = cv2.imread('test_images_directory/class_name/file_name.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # If the image is in BGR format
image = cv2.resize(image, (224, 224))  # Resize to match the model's input size
image_float_np = np.float32(image) / 255
print(f"image: {image.shape}")

In [ ]:
# Select the last layer of each base model
effici_model = new_effici_model
effici_target_layers = [effici_model.simclr.encoder.conv_layers[0][7][1]]
effici_model.eval()

resnet_model = new_resnet_model
resnet_target_layers = [resnet_model.simclr.encoder.conv_layers[7][2].bn3]
resnet_model.eval()

mobile_model = new_mobile_model
mobile_target_layers = [mobile_model.simclr.encoder.conv_layers[0][16][1]]
mobile_model.eval()

# Build Grad-CAM explainer
effici_gradcam = GradCAM(effici_model, effici_target_layers, use_cuda=torch.cuda.is_available())
resnet_gradcam = GradCAM(resnet_model, resnet_target_layers, use_cuda=torch.cuda.is_available())
mobile_gradcam = GradCAM(mobile_model, mobile_target_layers, use_cuda=torch.cuda.is_available())

# Generate the Grad-CAM heatmap
target_class = [ClassifierOutputTarget(1)]  # Set the target class if needed

# Transform the selected data into tensor
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
])
input_tensor = transform(image)
input_tensor = input_tensor.to(DEVICE)
input_tensor = input_tensor.unsqueeze(0)

# Use the input data to make prediction
resnet_prediction = resnet_model(input_tensor)
effici_prediction = effici_model(input_tensor)
mobile_prediction = mobile_model(input_tensor)

_, resnet_prediction_result = torch.max(resnet_prediction, 1)
_, effici_prediction_result = torch.max(effici_prediction, 1)
_, mobile_prediction_result = torch.max(mobile_prediction, 1)

# Generate corresponding heatmap
effici_heatmap = effici_gradcam(input_tensor=input_tensor, targets = target_class)[0, :]
resnet_heatmap = resnet_gradcam(input_tensor=input_tensor, targets = target_class)[0, :]
mobile_heatmap = mobile_gradcam(input_tensor=input_tensor, targets = target_class)[0, :]


In [ ]:
# Compare the prediction of base model with ground truth label
effici_cam_image = show_cam_on_image(image_float_np, effici_heatmap, use_rgb=True)
resnet_cam_image = show_cam_on_image(image_float_np, resnet_heatmap, use_rgb=True)
mobile_cam_image = show_cam_on_image(image_float_np, mobile_heatmap, use_rgb=True)

print(f"ResNet Prediction class: {resnet_prediction_result}")
print(f"EfficientNet Prediction class: {effici_prediction_result}")
print(f"MobileNet Prediction class: {mobile_prediction_result}")

# Plot the original image with the heatmap overlay
plt.figure(figsize=(10, 10))

plt.subplot(1, 3, 1)
plt.imshow(cv2.resize(cv2.cvtColor(cv2.imread('test_images_directory/class_name/file_name.jpg'), cv2.COLOR_BGR2RGB), (224, 224)))
plt.title('Ground Truth Image')

plt.subplot(1, 3, 2)
plt.imshow(Image.fromarray(resnet_cam_image), cmap='viridis')
plt.title('Resnet HeatMap')

plt.subplot(1, 3, 3)
plt.imshow(Image.fromarray(effici_cam_image), cmap='viridis')
plt.title('EfficientNet HeatMap')

plt.tight_layout()
plt.show()

plt.figure(figsize=(6, 6))

plt.subplot(1, 1, 1)
plt.imshow(Image.fromarray(mobile_cam_image), cmap='viridis')
plt.title('MobileNet HeatMap')


plt.tight_layout()
plt.show()

#### **Combined Grad-CAM visualisation**

Use multiple Grad-CAM heatmaps to analyse the combined model attention and generate visualisation heatmaps.

In [ ]:
# Combine multiple Grad-CAM heatmaps
def normalize_heatmap(heatmap):
    heatmap = np.maximum(heatmap, 0)
    heatmap = heatmap / np.max(heatmap)
    return heatmap

def resize_heatmap(heatmap, target_size):
    return cv2.resize(heatmap, target_size)

def combine_heatmaps(heatmaps, method='average'):
    if method == 'average':
        combined_heatmap = np.mean(heatmaps, axis=0)
    elif method == 'sum':
        combined_heatmap = np.sum(heatmaps, axis=0)
    combined_heatmap = normalize_heatmap(combined_heatmap)
    return combined_heatmap

def superimpose_heatmap_on_image(image, heatmap, alpha=0.4, colormap=cv2.COLORMAP_JET):
    heatmap = cv2.applyColorMap(np.uint8(255 * heatmap), colormap)
    superimposed_image = cv2.addWeighted(heatmap, alpha, image, 1 - alpha, 0)
    return superimposed_image

# Example usage
image2 = cv2.cvtColor(cv2.resize(cv2.imread('test_images_directory/class_name/file_name.jpg'),(224, 224)), cv2.COLOR_BGR2RGB)
heatmaps2 = [resnet_heatmap, mobile_heatmap, effici_heatmap]  
heatmaps2 = [normalize_heatmap(hm) for hm in heatmaps2]
heatmaps2 = [resize_heatmap(hm, (image.shape[1], image.shape[0])) for hm in heatmaps2]

combined_heatmap = combine_heatmaps(heatmaps2)
superimposed_image = superimpose_heatmap_on_image(image2, combined_heatmap)

cv2.imwrite('Combined_image.jpg', superimposed_image)

In [ ]:
plt.imshow(superimposed_image)
plt.title('Combined HeatMap')
plt.show()

#### **Deep SHAP visualisation**

Use deep SHAP explainer to analyse the model attention and generate visualisation heatmaps.

In [ ]:
# Get one batch of data from the data loader
images, targets = next(iter(test_dl)) # Acquire the target test dataloader from data preprocessing file

# Set a batch size for output sample
Batch_SIZE = 32
background_images = images[:Batch_SIZE].to(DEVICE)
background_targets = targets[:Batch_SIZE].numpy()

test_images = images[Batch_SIZE:].to(DEVICE)
test_targets = targets[Batch_SIZE:].numpy()

In [ ]:
def show_attributions(model):
    # Predict the probabilities of the digits using the test images
    output = model(test_images)
    # Get the index of the max log-probability
    pred = output.max(1, keepdim=True)[1] 
    # Convert to numpy only once to save time
    pred_np = pred.cpu().numpy() 

    expl = shap.DeepExplainer(model, background_images)

    for i in range(0, len(test_images)):
        ti = test_images[[i]]
        sv = expl.shap_values(ti)
        sn = [np.swapaxes(np.swapaxes(s, 1, -1), 1, 2) for s in sv]
        tn = np.swapaxes(np.swapaxes(ti.cpu().numpy(), 1, -1), 1, 2)

        shap.image_plot(sn, tn, show=False)

        # Prepare to augment the plot
        fig = plt.gcf()
        allaxes = fig.get_axes()

        # Show the actual/predicted class
        allaxes[0].set_title('Actual: {}, pred: {}'.format(
            test_targets[i], pred_np[i][0]))

        # Show the probability of each class
        prob = output[i].detach().cpu().numpy()
        for x in range(1, len(allaxes)-1):
            allaxes[x].set_title('{:.2%}'.format(prob[x-1]), fontsize=14)
        plt.show()

show_attributions(new_effici_model)